<a href="https://colab.research.google.com/github/Victoooooor/ErgoChairML/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Environment

In [1]:
#@title
!apt-get install libmagic-dev
!pip install git+https://github.com/Victoooooor/ErgoChairML.git

!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

# LOGIN

In [2]:
from google.colab import auth
auth.authenticate_user()

# Load trained model

In [3]:
bucket_name = 'ergo_chair_ml'
!gsutil -m cp -r gs://{bucket_name}/* ./

Copying gs://ergo_chair_ml/movenet_thunder.tflite...
Copying gs://ergo_chair_ml/masked_checkpoints/ckpt-7.index...
Copying gs://ergo_chair_ml/masked_checkpoints/ckpt-9.data-00000-of-00001...
Copying gs://ergo_chair_ml/masked_checkpoints/checkpoint...
Copying gs://ergo_chair_ml/masked_checkpoints/ckpt-7.data-00000-of-00001...
Copying gs://ergo_chair_ml/masked_checkpoints/ckpt-8.data-00000-of-00001...
Copying gs://ergo_chair_ml/masked_checkpoints/ckpt-8.index...
Copying gs://ergo_chair_ml/skeleton_checkpoints/checkpoint...
Copying gs://ergo_chair_ml/skeleton_checkpoints/ckpt-5.index...
Copying gs://ergo_chair_ml/masked_checkpoints/ckpt-9.index...
Copying gs://ergo_chair_ml/skeleton_checkpoints/ckpt-6.data-00000-of-00001...
Copying gs://ergo_chair_ml/skeleton_checkpoints/ckpt-6.index...
Copying gs://ergo_chair_ml/skeleton_checkpoints/ckpt-7.data-00000-of-00001...
Copying gs://ergo_chair_ml/skeleton_checkpoints/ckpt-7.index...
\ [14/14 files][  3.2 GiB/  3.2 GiB] 100% Done  71.2 MiB/s ETA 

# Init

In [4]:
from gen_chair import pix2pix

from gen_chair import coco
from gen_chair.gen_multi import Preprocess
from gen_chair.gen_pose import Preprocess as Preprocess2
from gen_chair import pix2pix

from google.colab import files
import shutil
import os
import tqdm
import cv2
import tensorflow as tf
from PIL import Image
import numpy as np


class InferenceConfig(coco.CocoConfig):
  # Set batch size to 1 since we'll be running inference on
  # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

def Infer(preproc, generator, origin):  

  seg = preproc(origin)
  origin = tf.cast(tf.image.resize_with_pad(origin, 256, 256), dtype=tf.int32)
  
  if seg is not None:
    seg  = tf.keras.preprocessing.image.img_to_array(seg)

  else:
    seg = origin

  seg = tf.expand_dims(seg, axis=0)
  seg = tf.cast(tf.image.resize_with_pad(seg, 256, 256), dtype=tf.int32)

  gen = generator(seg ,training=True)
  gen = tf.keras.utils.array_to_img(gen[0])

  seg = tf.keras.utils.array_to_img(seg[0])
    

  
  origin = tf.keras.utils.array_to_img(origin)
  dst = Image.new('RGB', (origin.width + seg.width + gen.width, origin.height))
  dst.paste(origin, (0, 0))
  dst.paste(seg, (origin.width, 0))
  dst.paste(gen, (origin.width+seg.width, 0))
  return dst

imgdr = 'data/'

prep = Preprocess(InferenceConfig)
prep2 = Preprocess2()
cpdir_mask = 'masked_checkpoints'
masked = pix2pix(cpdir_mask)
masked.loadcp()

cpdir_ske = 'skeleton_checkpoints'
skeleton = pix2pix(cpdir_ske)
skeleton.loadcp()

... done downloading pretrained model!

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE

Instructions for updating:
Use fn_output_signature instead


Restored from masked_checkpoints/ckpt-9
Restored from skeleton_checkpoints/ckpt-7


# Upload Files

The tab under will prompt to upload local files,
Support Format:
  most image format
  MP4, AVI
  zip

In [ ]:
uploaded = files.upload()
img_suffix = ('.jpg', '.png', '.ico', '.gif', '.jpeg')
vid_suffix = ('.avi','.mp4')
filenames = uploaded.keys()

for filename in filenames:
  if filename.endswith('.zip'):
    !unzip {filename} -d "data"
    os.remove(filename)
  elif filename.endswith(img_suffix) or filename.endswith(vid_suffix):
    os.makedirs("data/", exist_ok=True)
    shutil.move(filename, "data/")

# Process & Download

In [ ]:
image_names = sorted([n for n in os.listdir(imgdr) if not n.startswith('.')])

outdr1 = 'out_seg'
!mkdir -p {outdr1}
outdr2 = 'out_ske'
!mkdir -p {outdr2}
audiofile = '_sound.mp3'

for image_name in tqdm.tqdm(image_names):
  image_path = os.path.join(imgdr, image_name)
  if image_path.endswith(img_suffix):
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image)
    maskout = Infer(prep.img_seg, masked.generator, image)
    skeletonout = Infer(prep2.img_seg, skeleton.generator, image)
    maskout.save(os.path.join(outdr1, image_name))
    skeletonout.save(os.path.join(outdr2, image_name))
  elif image_path.endswith(vid_suffix):
    print('video',image_path)
    video = cv2.VideoCapture(image_path)
    if not video.isOpened():
      print('video does not exist')
      continue
    fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_num = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = 256*3
    frame_height = 256
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    name1 = os.path.join(outdr1, image_name)
    name2 = os.path.join(outdr2, image_name)
    video_writer1 = cv2.VideoWriter(name1,fourcc,fps,(frame_width,frame_height))
    video_writer2 = cv2.VideoWriter(name2,fourcc,fps,(frame_width,frame_height))
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
    counter =0
    while True:
      ret, frame = video.read()
      if ret == True:
          tfframe= tf.convert_to_tensor(frame)
          mframe = Infer(prep.img_seg, masked.generator, tfframe)
          sframe = Infer(prep2.img_seg, skeleton.generator, tfframe)
          m_cv2 = cv2.cvtColor(np.array(mframe), cv2.COLOR_RGB2BGR)
          s_cv2 = cv2.cvtColor(np.array(sframe), cv2.COLOR_RGB2BGR)
          video_writer1.write(m_cv2)
          video_writer2.write(s_cv2)
          counter +=1
      if ret == False:
          break
    video.release()
    video_writer1.release()
    video_writer2.release()
    cv2.destroyAllWindows()

!zip -r file.zip {outdr1} {outdr2}
files.download('file.zip')

In [7]:
!rm -rf {outdr1} {outdr2} {imgdr}